This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5) and depends on Magma (tested using version
2.26-9).

In this notebook, we identify trigonal curves of genus 7 with Maroni invariant 1 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 5 minutes for completion. 

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of $\mathbf{P}^1 \times \mathbf{P}^2$.

In [2]:
F = GF(2)
P.<x0,x1,y0,y1,y2> = PolynomialRing(F, 5)

In [3]:
#S1 = [vector(F,t) for t in itertools.product(F, repeat=2) if not all(i==0 for i in t)]
S1 = [vector(t) for t in ProjectiveSpace(F, 1)]
#S2 = [vector(F,t) for t in itertools.product(F, repeat=3) if not all(i==0 for i in t)]
S2 = [vector(t) for t in ProjectiveSpace(F, 2)]
for v in S1 + S2:
    v.set_immutable()
S = list(itertools.product(S1, S2))
len(S)

21

Construct the group $\mathrm{GL}(2, \mathbb{F}_2) \times \mathrm{GL}(3, \mathbb{F}_2)$ as a subgroup of $\mathrm{GL}(5, \mathbb{F}_2)$.

In [4]:
G = GL(5, F)
G1 = GL(2, F)
G2 = GL(3, F)
l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(3)], subdivide=False) for g in G1.gens()] + \
    [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in G2.gens()]
G0 = G.subgroup(l0)
G0.order()

1008

Use an orbit lookup tree to find $\mathrm{GL}(2,\mathbb{F}_2) \times \mathrm{GL}(3, \mathbb{F}_2)$-orbit representatives for 7-tuples of $\mathbb{F}_2$-points in $\mathbb{P}^1 \times \mathbb{P}^2$. We exclude tuples in which some 4 points have the same image in $\mathbb{P}^1$.

In [5]:
def apply_group_elem(g, x):
    g1 = g.matrix().submatrix(nrows=2,ncols=2)
    g2 = g.matrix().submatrix(row=2,col=2)
    v1 = g1*x[0]
    v2 = g2*x[1]
    v1.set_immutable()
    v2.set_immutable()
    return (v1, v2)

In [6]:
def stabilizer(x):
    G1 = vec_stab(Matrix(F, x[0]), transpose=True)
    G2 = vec_stab(Matrix(F, x[1]), transpose=True)
    l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(3)], subdivide=False) for g in G1.gens()] + \
        [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in G2.gens()]
    return GL(5, F).subgroup(l0)

In [7]:
def optimized_rep(g):
    return g.matrix()

In [8]:
def forbid(vecs, easy=False):
    if len(vecs) == 4:
        if len(set([x[0] for x in vecs])) == 1:
            return True    
    return False

In [9]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'forbid': forbid}
tree = build_orbit_tree(G0, S, 7, methods, verbose=False)
l = green_nodes(tree, 7)
len(l)

113

For each orbit of 7-tuples, find complete intersections of type $(1,1), (3,3)$ passing through these points. 
We use the following facts.
- The first hypersurface is a $\mathbf{P}^1$-bundle over $\mathbf{P}^1$.
- The second hypersurface is only specified modulo the ideal generated by the first.
- Given the first hypersurface, every $\mathbb{F}_2$-point of $\mathbf{P}^1 \times \mathbf{P}^2$ lying on it but not on the second defines an *affine* condition on the third hypersurface. We can thus find candidates for the second hypersurface by solving a system of linear equations.

In [10]:
monos11 = [prod(i) for i in itertools.product([x0,x1],[y0,y1,y2])]
monos22 = [prod(i) for i in itertools.product([prod(j) for j in itertools.combinations_with_replacement([x0,x1],2)], 
                                              [prod(j) for j in itertools.combinations_with_replacement([y0,y1,y2],2)])]
monos33 = [prod(i) for i in itertools.product([prod(j) for j in itertools.combinations_with_replacement([x0,x1],3)], 
                                              [prod(j) for j in itertools.combinations_with_replacement([y0,y1,y2],3)])]

In [11]:
len(monos11), len(monos33)

(6, 40)

In [12]:
coords1 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos11)) for x in S}
coords2 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos33)) for x in S}

In [13]:
def redundancy(v1, F=F, monos11=monos11, monos22=monos22, monos33=monos33):
    gens1 = sum(v1[i]*monos11[i] for i in range(6))
    return [vector(F, ((gens1*y).coefficient(mu) for mu in monos33)) for y in monos22]

In [14]:
l2 = []
for vecs in l:
    V1 = Matrix([coords1[x] for x in vecs]).right_kernel()
    for v1 in V1:
        # Check that this vector defines a smooth (1,1)-hypersurface.
        if v1[:3] == 0 or v1[3:] == 0 or v1[:3] == v1[3:]:
            continue
        pts = [x for x in S if coords1[x]*v1 == 0]
        perp = Matrix([coords2[x] for x in pts])
        target = vector(F, (0 if x in vecs else 1 for x in pts))
        for v2 in solve_right_iterator(perp, target, redundancy, v1):
            l2.append([sum(v1[i]*monos11[i] for i in range(6)),
                sum(v2[i]*monos33[i] for i in range(40))])
len(l2)

40960

Enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=2,3$ using commutative algebra.

In [15]:
def count_by_ideal(gens, n):
    J = P.ideal(gens + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 2^(3*n) - 2^(2*n) + 1) // (2^n-1)^2

In [16]:
l3 = [gens for gens in l2 if count_by_ideal(gens, 2) == 15]
len(l3)

4664

In [17]:
l4 = [gens for gens in l3 if count_by_ideal(gens, 3) == 7]
len(l4)

8

Close out this case.

In [18]:
I1 = P.ideal([x0,x1])
I2 = P.ideal([y0,y1,y2])
CR = magma.CoxRing(P, [I1, I2], [[1,1,0,0,0],[0,0,1,1,1]], [])
proj = CR.ToricVariety()

In [19]:
curves = {(7,15,7): l4}
closeout(curves, X=proj)

Number of curves found: 0
Number of isomorphism classes found: 0
No covers found in this case!
Total time: 167 seconds
